In [ ]:
from test_floris_sr_utils import optimize_florissr
import numpy as np

layout_4wt = ([0.0, 250.0, 0.0, 250.0], [0.0, 0.0, 250.0, 250.0])
layout_8lhs = np.load("data/layouts/lhs_env_nt8_md150_wb750x750_turbine_layout.npy")
layout_16lhs = np.load("data/layouts/lhs_env_nt16_md75_wb1500x1500_turbine_layout.npy")

# optimize_florissr(layout_4wt, "serial_refine/4wt", "4 symmetric")
# optimize_florissr(layout_8lhs, "serial_refine/8lhs", "8 LHS")
optimize_florissr(layout_16lhs, "serial_refine/16lhs", "16 LHS")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

layout_names = ["serial_refine/4wt", "serial_refine/8lhs", "serial_refine/16lhs"]
layout_title_names = ["4 symmetric", "8 LHS", "16 LHS"]

for layout_name, layout_title_name in zip(layout_names, layout_title_names):
    geo_farm_power = np.load(f"data/{layout_name}_geo_farm_power.npy")
    df_opt_sr = np.load(f"data/{layout_name}_sr_farm_power.npy")
    df_opt_scipy = np.load(f"data/{layout_name}_scipy_farm_power.npy")
    df_opt_baseline = np.load(f"data/{layout_name}_baseline_farm_power.npy")

    total_power_uplift_geo = np.sum(geo_farm_power - df_opt_baseline)
    total_power_uplift_sr = np.sum(df_opt_sr - df_opt_baseline)
    total_power_uplift_scipy = np.sum(df_opt_scipy - df_opt_baseline)

    fig, axarr = plt.subplots(1,1,figsize=(5,5))

    ax = axarr
    ax.bar(
        [0, 1, 2],
        [
            total_power_uplift_geo / total_power_uplift_scipy,
            total_power_uplift_sr / total_power_uplift_scipy,
            1.0,
        ],
        color=['m', 'r', 'g'],
    )
    ax.set_xticks([0, 1, 2])
    ax.set_xticklabels(['Geometric', 'Serial Refine', 'SLSQP'])
    ax.set_ylabel('Normalized Power Gain')
    ax.grid(True)
    plt.title(f"Normalized power gain for layout {layout_title_name}")
    plt.savefig(f"./figures/used/{layout_name}_power_gain.png")
    plt.show()

In [ ]:
from serial_refine_agent import plot_yaw_angles

for layout in ["serial_refine/4wt", "serial_refine/8lhs", "serial_refine/16lhs"]:
    optimal_yaws = np.load(f"data/{layout}/yaw_angles_opt_sr.npy")
    wind_directions = np.linspace(0, 360, 120)
    plot_yaw_angles("serial_refine", wind_directions, optimal_yaws)

# Rendering in the environment

In [ ]:
# Wind process that implements a sequence of wind speeds and directions
from wind_processes import SetSequenceWindProcess

In [ ]:
from env_utils import get_4wt_symmetric_env

env = get_4wt_symmetric_env(episode_length=100, privileged=False, action_representation="yaw", load_pyglet_visualization=True)
repeat = 10
middle_angle = 226
offset = 5
starting_time_offset = 40
wind_directions = [middle_angle-offset]*starting_time_offset
for i in range(offset*2):
    wind_directions.extend([middle_angle + (i - offset)]*repeat)
env.wind_process = SetSequenceWindProcess(wind_directions=wind_directions, wind_speeds=[8.]*len(wind_directions))
print(wind_directions)
# Get proportional controller
from serial_refine_agent import SR_ProportionalController

agent = SR_ProportionalController()

# Run the environment
done = False
imgs = []
obs = env.reset()
for i in range(100):
    wind_direction = env.wind_process.wind_direction
    current_yaws = env.yaws_from_wind

    action = agent.predict(wind_direction, current_yaws)
    obs, reward, terminated, truncated, info = env.step(action)

    env.render()
    imgs.append(env.render(mode='rgb_array'))
    if terminated or truncated:
        break

In [ ]:
from PIL import Image

ofs_top = 0
l = 300
imgs_pil = [Image.fromarray(img[ofs_top:ofs_top+l, ofs_top+7:ofs_top+l]) for img in imgs]

# Save gif
imgs_pil[starting_time_offset].save("data/serial_refine/critical_angle.gif", save_all=True, append_images=imgs_pil[starting_time_offset+1:], loop=0, duration=100)